In [1]:
#r "nuget: Testcontainers"
#r "nuget: Testcontainers.Postgresql"
#r "nuget: Npgsql"

## ![img](https://testcontainers.com/modules/postgresql//logo.svg) Setup infrastructure


To use PostgreSQL as a vector database, you can leverage the `vector` extension provided by the `pgvector/pgvector` image. This extension allows you to store and query vector embeddings directly in the database.

By using PostgreSQL as a vector database, you can perform high-level operations on vector data, such as similarity search, nearest neighbor search, and clustering. This enables you to efficiently work with large-scale vector datasets and build applications that require advanced vector operations.

For more information on using PostgreSQL as Test Container, you can refer to the [Testcontainers PostgreSQL module documentation](https://testcontainers.com/modules/postgresql/).

In [2]:
using Testcontainers.PostgreSql;
using Npgsql;

var db =  new PostgreSqlBuilder()
    .WithImage("pgvector/pgvector:pg16")
    .WithPortBinding(5432, 5432)
    .WithDatabase("memory-db")
    .WithUsername("postgres")
    .WithPassword("postgres")
    .WithReuse(true)
    .WithVolumeMount("memory-db-volume", "/var/lib/postgresql/data")
    .Build();

await db.StartAsync();

var connectionString = db.GetConnectionString();

connectionString

### Configure 'vector' extension

In [26]:
try
{
    using (var conn = new NpgsqlConnection(connectionString))
    {
        conn.Open();

        var sql = "CREATE EXTENSION vector";

        using var cmd = new NpgsqlCommand(sql, conn);

        var rowsAffected = cmd.ExecuteNonQuery();

        display("Extension created successfully");
    }
}
catch (PostgresException ex)
{
    display(ex.Message);
}

### Stop container

In [32]:
await db.StopAsync();

## ![asipire-logo](https://learn.microsoft.com/en-us/dotnet/aspire/assets/dotnet-aspire-logo-128.svg) Use Aspire Dashboard

<https://mcr.microsoft.com/en-us/product/dotnet/aspire-dashboard/tags>

Open: <http://localhost:18888>

In [38]:
using Testcontainers;
using DotNet.Testcontainers.Builders;

var aspireDashboard = new ContainerBuilder()
    .WithImage("mcr.microsoft.com/dotnet/aspire-dashboard:8.1.0")
    .WithPortBinding(18888, 18888)
    .WithPortBinding(18889, 18889)
    .WithEnvironment("DOTNET_DASHBOARD_UNSECURED_ALLOW_ANONYMOUS", "true")
    .WithWaitStrategy(
        Wait.ForUnixContainer().UntilHttpRequestIsSucceeded(r => r.ForPort(18888))
    )
    .WithReuse(true)
    .Build();

await aspireDashboard.StartAsync();

var otelExporterEndpoint =
    $"http://localhost:{aspireDashboard.GetMappedPublicPort(18889)}";

otelExporterEndpoint